In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import datetime
# External dependencies
# pip install geopy graphviz folium
import folium
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

#Creating neural network

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, GRU, Embedding, CuDNNLSTM, BatchNormalization
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import TensorBoard

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
trip_weather_station_ID = pickle.load(open("/content/drive/My Drive/Data/Bikes_data/data/trip_weather_station_ID.pickle", "rb"))

weather_station_dup = pickle.load(open("/content/drive/My Drive/Data/Bikes_data/data/weather_station_dup.pickle", "rb"))
station_data_orig = pd.read_csv("/content/drive/My Drive/Data/Bikes_data/data/station_data.csv")
trip_data_orig = pd.read_csv('/content/drive/My Drive/Data/Bikes_data/data/trip_data.csv')

In [0]:
station_data_dup = station_data_orig.copy()
trip_data_dup = trip_data_orig.copy()

In [0]:
station_data_dup.City.value_counts()

San Francisco    39
San Jose         16
Redwood City      9
Mountain View     7
Palo Alto         5
Name: City, dtype: int64

#Name the city of interest to check the bike availability

In [0]:
City = 'Redwood City'


In [0]:
map_city = station_data_dup[station_data_dup['City'] == City]


In [0]:
net_change_bikes_result = []
for station_id in map_city['Id']:
  def number_of_bikes_station_id (station_id):
        trip_data_dup_index = trip_data_orig.copy() # copying the dataframe
        trip_data_dup_index['Start Date'] = pd.to_datetime (trip_data_dup_index['Start Date'])
        trip_data_dup_index['End Date'] = pd.to_datetime (trip_data_dup_index['End Date'])
        #trip_data_dup_index.drop ('Trip ID', axis = 1, inplace = True) # droping the column \"TripId\" because it doesnt give much information
        trip_data_dup_index.set_index ('Start Date', inplace = True) # setting the index as \"Start date\" for the column\n",
        start_station = trip_data_dup_index[trip_data_dup_index["Start Station"]==station_id].resample('1H').count()
        #"#start_station.reset_index (inplace = True)\n",
        #print (start_station['Start Station'].head(10))\n",
        end_station = trip_data_dup_index[trip_data_dup_index["End Station"]==station_id].resample('1H').count()
        #end_station.reset_index (inplace = True)\n",
        #print(end_station['End Station'].head(10))\n",
        balance_station = pd.concat ([start_station['Start Station'],end_station['End Station']] , axis =1)
        balance_station.fillna(value = 0, inplace = True)
        #balance_station.head(10)\n",
        balance_station['net_change_bikes']= balance_station['End Station']-balance_station['Start Station']
        balance_station['total_bikes_change'] = balance_station['net_change_bikes'].cumsum()
        balance_station['total_bikes_pct_change'] = balance_station['total_bikes_change'].pct_change()
        balance_station.replace ([np.inf, -np.inf], np.nan , inplace = True)
        
        #Dock_count = station_data[station_data['Id'] == station_id]['Dock Count']
        #    Dock_count = Dock_count.values\n",
        #     docks = no_of_bikes+Dock_count\n",
        #     print (docks.min(), docks.max())\n",
        return balance_station
  def trip_weather_station_merge_ID(station_id):
        trip_start_end_station = number_of_bikes_station_id (station_id)
        trip_start_end_station.reset_index(inplace = True) #Remove the index from this
        trip_start_end_station["Date"] = trip_start_end_station["Start Date"].dt.date # creating a new column of Date
        trip_start_end_station.head()
        trip_start_end_station['Date']= pd.to_datetime (trip_start_end_station['Date']) # Converting that column of Date into pandas date format 
        weather_station = pickle.load (open("/content/drive/My Drive/Data/Bikes_data/data/weather_station_dup.pickle", "rb")) # Loading the data of weather_station merger dataframe with pickle
        weather_station['Date']= pd.to_datetime(weather_station['Date'])  # Changing the weather dataframe "Date" column into datetime so that we can merge with start_end_station
        weather_station_id = weather_station[weather_station["Id"] == station_id] # Taking only the values of selected station id
        #weather_station_id = weather_station.copy()
        weather_station_id.fillna(method = 'ffill', inplace = True)
        print ("Weather_station_id shape is" , weather_station_id.shape)
        print ("start_end_station shape is" , trip_start_end_station.shape)
        trip_start_end_station.reset_index( inplace = True)
        weather_station_id.reset_index ( inplace = True)
        trip_weather_station_ID =pd.merge (trip_start_end_station, weather_station_id, on = "Date", how = 'outer')
        print ("The number of rows of the merger should be same as start_end_station length of rows :" ,trip_weather_station_ID.shape)
        trip_weather_station_ID.fillna(method = 'ffill', inplace = True)
        return trip_weather_station_ID
  trip_weather_station_merge = trip_weather_station_merge_ID(station_id)
  #Creating time coumn
  trip_weather_station_ID['Time'] = trip_weather_station_ID['Start Date'].dt.time   # Adding "Time" column to the dataframe, which gives time on the particular day
  trip_weather_station_ID['dayofweek'] = trip_weather_station_ID['Start Date'].dt.dayofweek # Adding "dayofweek" column to the dataframe, which gives information about the day
  trip_weather_station_ID['month'] = trip_weather_station_ID['Start Date'].dt.month   # Adding "month" column to the dataframe, which gives information about the day
  #Dropping the columns that are not necessary
  trip_weather_station_ID_drop = trip_weather_station_ID[['Start Date', 'Max TemperatureF', 'Max Wind SpeedMPH', 'Min TemperatureF', 'dayofweek', 'month' ,'total_bikes_pct_change','total_bikes_change' , 'net_change_bikes']]
  corrmat = trip_weather_station_ID_drop.corr(method = 'spearman')
  #   plt.subplots(figsize = (12,9))
  #   sns.heatmap (corrmat, vmax = 0.9, square = True)
  #Number of columns to be dropped "Date", "total_bikes_change", "total_bikes_pct_change", "Start Date", 
  #Setting up the index column
  trip_weather_station_ID_drop.set_index('Start Date', inplace = True)
  trip_weather_station_ID_drop.fillna(method = 'ffill', inplace = True)
  trip_weather_station_ID_drop.drop(['net_change_bikes', 'total_bikes_pct_change'], axis = 1, inplace = True)
  #Shifting the target
  # The data is shifted depending on number of time steps we want to shift the data. In this case we would like to predict the data for 1 day, and the data is resample for an hour. 
  # So the data is shifted by 24 hours
  # Prediction column
  #We want to predict the future in order to do that we have to shift the target column into future

  shift_days = 1
  shift_time = shift_days * 24 # shift time hours The idea is to predict the movement of the bikes for next day
  shift_steps = 20 # since the data registered doesn't contain full 24 hours cycle of the day due to the nature of the data, hence, it is input manually.
  #Creating a new data frame with time shifted data
  ## Note the negative time shift
  target = trip_weather_station_ID_drop['total_bikes_change'].shift(-shift_steps)
  trip_weather_station_ID_drop.drop('total_bikes_change', axis = 1, inplace = True)
  #Numpy Arrays for putting into neural network
  
  x_data = trip_weather_station_ID_drop.values[0 : -shift_steps]
  
  y_data = target.values [:-shift_steps] # Output signals
  
  #Splitting the data between training and validation
  
  num_data = x_data.shape[0]

  train_split = 0.8
  #This is the number of observations in the training set
  num_train = int (train_split * num_data)
  num_test = num_data-num_train
  
  # splitting
  x_train = x_data[0:num_train]
  x_test = x_data[num_train:]
  num_x_signals = x_data.shape[1]
  y_data= y_data.reshape(-1,1)
  #Number of output signals
  
  num_y_signals = y_data.shape[1]
  y_train = y_data[0:num_train]
  y_test = y_data[num_train:]
  
  #The neural networks works best when the values are between -1 and 1, so to get optimum result we need to scale the data before entering into neural network. we use scikit preprocessing for this
  
  x_scaler = MinMaxScaler()
  
  #We then detect the range of values from training data and scale the training data
  
  x_train_scaled = x_scaler.fit_transform(x_train)
  
  #We use the same scaler objects in the input signal for the test sets
  
  x_test_scaled = x_scaler.transform(x_test)
  
  #We create seperate scalar object for the test set
  
  y_scaler = MinMaxScaler()
  y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1,1))
  y_test_scaled = y_scaler.transform(y_test.reshape(-1,1))
  
  #Instead of training the RNN at all sequences on the complete sequences , we will use the following function to create a batch of shorter sub sequences picked at random from the training data
  
  def batch_generator (batch_size, sequence_length):
  # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, sequence_length, num_x_signals)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        # Allocate a new array for the batch of output-signals.
        y_shape = (batch_size, sequence_length, num_y_signals)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = np.random.randint(num_train - sequence_length)
            
            # Copy the sequences of data starting at this index.
            x_batch[i] = x_train_scaled[idx:idx+sequence_length]
            y_batch[i] = y_train_scaled[idx:idx+sequence_length]
        
        yield (x_batch, y_batch)
  batch_size = 90 # Intializing the batch size
  sequence_length = 20*7 # 7 days of the data
  
  #We then create a batch generator
  
  generator = batch_generator(batch_size=batch_size,
                            sequence_length=sequence_length)
  
  #We can then test the batch generator to see if it works
  x_batch, y_batch = next(generator)
  
  #Validation set for testing the mode accuracy so that model wont overfit
  
  validation_data = (np.expand_dims(x_test_scaled, axis=0),
                   np.expand_dims(y_test_scaled, axis=0))
  
  #CREATING THE NEURAL NETWORK RNN
  
  model = Sequential()
  
  # Applying GRU model to the data
  model.add(GRU(units=128, return_sequences=True,             
                input_shape=(None, num_x_signals,)))

  model.add(Dropout(0.3))

  model.add(GRU(units=128,return_sequences=True))
  model.add(Dropout(0.3))

  model.add(Dense(units=32, activation = 'relu'))
  model.add(Dropout(0.3))



  model.add(Dense(num_y_signals, activation='sigmoid'))
  #Optimizer
  
  optimizer = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
  
  #We then compile the Keras model so it is ready for training.
  
  warmup_steps = 50
  def loss_mse_warmup(y_true, y_pred):
      """
      Calculate the Mean Squared Error between y_true and y_pred,
      but ignore the beginning "warmup" part of the sequences.

      y_true is the desired output.
      y_pred is the model's output.
      """

      y_true_slice = y_true[:, warmup_steps:, :]
      y_pred_slice = y_pred[:, warmup_steps:, :]


      loss = tf.losses.mean_squared_error(labels=y_true_slice,
                                          predictions=y_pred_slice)
  
      loss_mean = tf.reduce_mean(loss)

      return loss_mean
   
  model.compile(loss='mse', optimizer=optimizer, metrics = ['mae'])
  #Train the RNN
  
  model.fit_generator(generator=generator,
                    epochs=3,
                    steps_per_epoch=100,
                    validation_data=validation_data
                    )
  result = model.evaluate(x=np.expand_dims(x_test_scaled, axis=0),
                        y=np.expand_dims(y_test_scaled, axis=0))
  #print (result)
  
  #Generate Predictions
  
  #x_train_scaled.shape
  x = np.expand_dims(x_train_scaled, axis=0)
  y_true_train = y_train
  # Use the model to predict the output-signals.
  y_pred_train = model.predict(x)
  
  #The output of the model is between 0 and 1. Do an inverse map to get back into the scale of the original dataset
  
  y_pred_rescaled = y_scaler.inverse_transform(y_pred_train[0])
  
  #PREDICTION ON TEST DATA
  x_test = np.expand_dims(x_test_scaled, axis=0)
  y_true_test = y_test
  # Use the model to predict the output-signals.
  y_pred_test = model.predict(x_test)
  
  #The output of the model is between 0 and 1. Do an inverse map to get back into the scale of the original dataset

  y_pred_rescaled_test = y_scaler.inverse_transform(y_pred_test[0])
  
  #Predicting the model current output of net change in bikes at given station
  #For doing this, the current datetime is taken from the computer is registered and with weather values picked up randomly from the dataset the net change in bikes at any goven station is predicted for current time.
  current_time = datetime.datetime.now() # Importing the time from the computer with pandas

  current_test = {'Max TemperatureF' : [78],     # Creating a list with randomly picked weather values from the dataset
                'Max Wind SpeedMPH' : [18],
                 'Min TemperatureF' : [55]
                  }


  df = pd.DataFrame(current_test)                     # Creating a dataframe
  date_range = pd.to_datetime('today')                  # Date of today no time included  
  time_range = pd.date_range("00:00", current_time, freq="1H")  # creating a time range from midnight today to currrent time 
  #df_time = pd.DataFrame (time_range)                       
  length_time = len(time_range)
  length_time
  df = pd.concat([df]*length_time)                # Since the weather is daiy forecast it stays same entire day, so rows are duplicated depending on the time range
  lenght_time_df = len(df)
  length_time = len(time_range)
  print (lenght_time_df)
  print (length_time)
  df['Date'] = time_range                       # Creating a column with date and time
  df['dayofweek'] = df['Date'].dt.dayofweek     # Calculating day of the week
  df['month']= df['Date'].dt.month              # Calculating month
  
  df.set_index('Date', inplace = True)      # Set DATE as index
  #Input the current datetime dataframe to model
  
  #Scaling the values of the dataframe
  
  df1 = df.values
  #x_test_scaled = x_scaler.transform(df.values)

  x_df_scaled = x_scaler.transform(df1)
  
  #Changing the dimensions and applying the model prediction
  
  x_df =  np.expand_dims(x_df_scaled, axis=0)
  # Use the model to predict the output-signals.
  y_pred_df = model.predict(x_df)
  
  #Inverse transforming the output
  
  y_pred_rescaled_df = y_scaler.inverse_transform(y_pred_df[0])
  
  #To measure the net change in bikes we need to know the previous hour values and current value, that is the reason that range is applied so that we can extract current value from previous.
  
  res = [x-y for x, y in zip (y_pred_rescaled_df,y_pred_rescaled_df[1:])]
  print ('Net change in bikes :' , res[-1] )   # The last value referes to the current value and that is printed out .
  net_change_bikes = res[-1].tolist()
  net_change_bikes_result.append(net_change_bikes)
  
net_change_bikes_result  

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


Weather_station_id shape is (365, 25)
start_end_station shape is (15823, 7)
The number of rows of the merger should be same as start_end_station length of rows : (15835, 33)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Epoch 1/3
100/100 [==============================] - 47s 466ms/step - loss: 0.0781 - mean_absolute_error: 0.2398 - val_loss: 0.1466 - val_mean_absolute_error: 0.3683
Epoch 2/3
100/100 [==============================] - 45s 449ms/step - loss: 0.0395 - mean_absolute_error: 0.1458 - val_loss: 0.0136 - val_mean_absolute_error: 0.0880
Epoch 3/3
1/1 [==============================] - 2s 2s/sample - loss: 0.1007 - mean_absolute_error: 0.2958
12
12
Net change in bikes : [0.20379639]


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


...

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


Weather_station_id shape is (365, 25)
start_end_station shape is (16012, 7)
The number of rows of the merger should be same as start_end_station length of rows : (16016, 33)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Epoch 1/3
100/100 [==============================] - 49s 492ms/step - loss: 0.0771 - mean_absolute_error: 0.2381 - val_loss: 0.1203 - val_mean_absolute_error: 0.3330
Epoch 2/3
100/100 [==============================] - 46s 462ms/step - loss: 0.0462 - mean_absolute_error: 0.1599 - val_loss: 0.0146 - val_mean_absolute_error: 0.1008
Epoch 3/3
1/1 [==============================] - 2s 2s/sample - loss: 0.0692 - mean_absolute_error: 0.2433
13
13
Net change in bikes : [-4.2410583]


[[0.20379638671875],
 [-4.219818115234375],
 [-11.271697998046875],
 [-10.52154541015625],
 [0.0443115234375],
 [-4.69818115234375],
 [-11.679412841796875],
 [1.942596435546875],
 [-4.241058349609375]]

In [0]:
# map_city = pd.DataFrame()
# #net_change_bikes_result = [-4.0464783, -0.31860352, -1.8529, -4.15972, -5.093322]
map_city ['net_change_bikes_result'] = net_change_bikes_result
map_city

,Id,Name,Lat,Long,Dock Count,City,net_change_bikes_result
14,21,Franklin at Maple,37.481758,-122.226904,15,Redwood City,[0.20379638671875]
15,22,Redwood City Caltrain Station,37.486078,-122.232089,25,Redwood City,[-4.219818115234375]
16,23,San Mateo County Center,37.488501,-122.231061,15,Redwood City,[-11.271697998046875]
17,85,San Mateo County Center,37.487616,-122.229951,15,Redwood City,[-10.52154541015625]
18,24,Redwood City Public Library,37.484219,-122.227424,15,Redwood City,[0.0443115234375]
19,25,Broadway at Main,37.486725,-122.225551,15,Redwood City,[-4.69818115234375]
20,86,Stanford in Redwood City,37.485370,-122.203288,15,Redwood City,[-11.679412841796875]
21,26,Redwood City Medical Center,37.487682,-122.223492,15,Redwood City,[1.942596435546875]
74,83,Mezes Park,37.491269,-122.236234,15,Redwood City,[-4.241058349609375]


In [0]:
map_city

[[0.20379638671875],
 [-4.219818115234375],
 [-11.271697998046875],
 [-10.52154541015625],
 [0.0443115234375],
 [-4.69818115234375],
 [-11.679412841796875],
 [1.942596435546875],
 [-4.241058349609375]]

In [0]:
city_coordinates = {'Palo Alto' : [37.4419, -122.1430],
                   'San Francisco' : [37.7749,-122.4194],
                   'Mountain View' : [37.3861, -122.0839],
                   'Redwood City' : [37.4852, -122.2364],
                   'San Jose' : [37.3382, -121.8863]
                   }

In [0]:
map_center = city_coordinates[City]
map_zoom = 14
station_map = folium.Map(location=map_center, zoom_start=map_zoom)
#folium.Marker([45, -30], popup='inline implicit popup').add_to(station_map)   ###
coordinates = map_city[['Lat', 'Long',  'Id', 'Name', 'net_change_bikes_result']]
for lat,long, Id, name , net_change_bikes_result in coordinates.values:
  #print (lat,long,Id)
  folium.CircleMarker([lat,long],
                    radius = 50, popup= f'Net change in bikes {net_change_bikes_result} Station ID {str(Id)}',
                    
                    color = '#dd0000', fill_color = '#dd0000',).add_to(station_map)
station_map

In [0]:
city_coordinates[City]

[37.4852, -122.2364]